In [ ]:
from longnetLM.config.config import *
from longnetLM.config.LongNetTokenizer import LongNetTokenizer

config_path = "longnetLM/config/config_0.1b"
vocab_path = "longnetLM/config/vocab.txt"
tokenizer = LongNetTokenizer(vocab_path)
config = LongnetConfig.from_pretrained(
    config_path,
    attention_type="DilatedAttention",
    vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    mask_token_id=tokenizer.mask_token_id,
    flash_attention=False,
    segment_length='[2048,4096]', 
    dilated_ratio='[1,2]', 
    )


{'activation_dropout': 0.0, 'activation_fn': 'gelu', 'attention_dropout': 0.0, 'attention_type': 'DilatedAttention', 'bos_token_id': 0, 'decoder_attention_heads': 12, 'decoder_ffn_embed_dim': 1280, 'decoder_layers': 12, 'decoder_normalize_before': True, 'deepnorm': False, 'dilated_ratio': [1, 2], 'drop_path_rate': 0.0, 'dropout': 0.0, 'embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_layers': 12, 'encoder_normalize_before': True, 'eos_token_id': 2, 'flash_attention': True, 'initializer_range': 0.02, 'layernorm_embedding': False, 'layernorm_eps': 1e-06, 'mask_token_id': 4, 'max_positions': 65536, 'max_rel_pos': 0, 'no_scale_embedding': True, 'normalize_output': True, 'num_attention_heads': 12, 'pad_token_id': 1, 'rel_pos_buckets': 0, 'segment_length': [2048, 4096], 'seq_parallel': False, 'subln': True, 'transformers_version': '4.44.2', 'vocab_size': 19, '_commit_hash': None, 'attn_implementation': None}
{}


In [ ]:

# build dataset 
from datasets import load_dataset

def pack(_tokenizer,max_length,padding="max_length",pad_to_multiple_of=None):
    def padseq(line):
        inputs=_tokenizer(line["seq"], max_length=max_length, truncation=True,padding=padding,pad_to_multiple_of=pad_to_multiple_of)
        return inputs
    return padseq
func=pack(tokenizer,26000,padding="max_length")
dataset_path=f"{YOUR_DATASET_PATH_HERE}"
train_ds = load_dataset("csv",data_files=f"{dataset_path}/sars-sars2_26k_train.csv").remove_columns("Unnamed: 0")
test_ds  = load_dataset("csv",data_files=f"{dataset_path}/sars-sars2_26k_test.csv" ).remove_columns("Unnamed: 0")
eval_ds  = load_dataset("csv",data_files=f"{dataset_path}/sars-sars2_26k_eval.csv" ).remove_columns("Unnamed: 0")
train_ds=train_ds.map(func,batched=True,num_proc=128)["train"].remove_columns("seq")
test_ds=test_ds.map(func,batched=True,num_proc=128)["train"].remove_columns("seq")
eval_ds=eval_ds.map(func,batched=True,num_proc=128)["train"].remove_columns("seq")

output_path=f"{YOUR_OUTPUT_PATH_HERE}"
perfix="SARS-SARS2_26k"
train_ds.save_to_disk(f"{output_path}/{perfix}/trainset", num_proc=128)
eval_ds.save_to_disk( f"{output_path}/{perfix}/evalset" , num_proc=128)
test_ds.save_to_disk( f"{output_path}/{perfix}/testset" , num_proc=128)


In [2]:
#encoder
from longnetLM.srcs.architecture.layers import *
from longnetLM.srcs.LongNetLM import LongNetEncoderLM,LongNetDecoderLM,LongNetEncoderDecoderLM
import torch
device="cpu"
x=torch.randint(0,7,(2,26000),device=device)
# emb=LongNetEmbeddings(config).to(dtype=torch.float16,device=device)
# encoder=Encoder(config).to(dtype=torch.float16,device=device)
# decoder=Decoder(config).to(dtype=torch.float16,device=device)
# model=LongNetEncoderLM(config).to(dtype=torch.float16,device=device)
# model=LongNetDecoderLM(config).to(dtype=torch.float16,device=device)
model=LongNetEncoderDecoderLM(config).to(dtype=torch.float16,device=device)


/Users/zhengyulong/opt/miniconda3/envs/ML/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
model

LongNetEncoderDecoderLM(
  (embedding): LongNetEmbeddings(
    (word_embeddings): Embedding(19, 768, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(65536, 768, padding_idx=1)
  )
  (encoder): Encoder(
    (dropout_module): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x EncoderLayer(
        (self_attn): DilatedAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (inner_attn_ln): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (dropout_module): Dropout(p=0.0, inplace=False)
        )
        (dropout_module): Dropout(p=0.0, inplace=False)
        (ffn): FeedForwardNetwork(
          (activation_dropout_module): Dropout(p=0.0, inplace=Fal

In [5]:
def p_count(m):
    ttp=0
    tp=0
    for p in m.parameters():
        c=p.numel()
        if p.requires_grad == True:
            ttp+=c
        tp+=c
    print(f"Total trainable parameters: {ttp}")
    print(f"Total parameters: {tp}")

In [6]:
p_count(model)

Total trainable parameters: 187547904
Total parameters: 187547904
